<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/038_Building_a_Simple_Agent_Frameworkipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Building a Simple Agent Framework

Now, we are going to put the components together into a **reusable agent class**. This class encapsulates the GAME components and provides a clean interface for running the agent loop. We can create different agents simply by changing the **Goals**, **Actions**, **Memory**, and **Environment** (GAME) without modifying the core loop.

In [ ]:
class Agent:
    def __init__(self,
                 goals: List[Goal],
                 agent_language: AgentLanguage,
                 action_registry: ActionRegistry,
                 generate_response: Callable[[Prompt], str],
                 environment: Environment):
        """
        Initialize an agent with its core GAME components
        """
        self.goals = goals
        self.generate_response = generate_response
        self.agent_language = agent_language
        self.actions = action_registry
        self.environment = environment

    def construct_prompt(self, goals: List[Goal], memory: Memory, actions: ActionRegistry) -> Prompt:
        """Build prompt with memory context"""
        return self.agent_language.construct_prompt(
            actions=actions.get_actions(),
            environment=self.environment,
            goals=goals,
            memory=memory
        )

    def get_action(self, response):
        invocation = self.agent_language.parse_response(response)
        action = self.actions.get_action(invocation["tool"])
        return action, invocation

    def should_terminate(self, response: str) -> bool:
        action_def, _ = self.get_action(response)
        return action_def.terminal

    def set_current_task(self, memory: Memory, task: str):
        memory.add_memory({"type": "user", "content": task})

    def update_memory(self, memory: Memory, response: str, result: dict):
        """
        Update memory with the agent's decision and the environment's response.
        """
        new_memories = [
            {"type": "assistant", "content": response},
            {"type": "user", "content": json.dumps(result)}
        ]
        for m in new_memories:
            memory.add_memory(m)

    def prompt_llm_for_action(self, full_prompt: Prompt) -> str:
        response = self.generate_response(full_prompt)
        return response

    def run(self, user_input: str, memory=None, max_iterations: int = 50) -> Memory:
        """
        Execute the GAME loop for this agent with a maximum iteration limit.
        """
        memory = memory or Memory()
        self.set_current_task(memory, user_input)

        for _ in range(max_iterations):
            # Construct a prompt that includes the Goals, Actions, and the current Memory
            prompt = self.construct_prompt(self.goals, memory, self.actions)

            print("Agent thinking...")
            # Generate a response from the agent
            response = self.prompt_llm_for_action(prompt)
            print(f"Agent Decision: {response}")

            # Determine which action the agent wants to execute
            action, invocation = self.get_action(response)

            # Execute the action in the environment
            result = self.environment.execute_action(action, invocation["args"])
            print(f"Action Result: {result}")

            # Update the agent's memory with information about what happened
            self.update_memory(memory, response, result)

            # Check if the agent has decided to terminate
            if self.should_terminate(response):
                break

        return memory




## 🧠 What Is `AgentLanguage`?

The `AgentLanguage` class is a **core abstraction** responsible for:

* 📝 **Formatting the prompt** that gets sent to the LLM
* 📦 **Parsing the model’s response** into structured tool calls

It acts as the “language layer” that determines how your agent **talks to** and **interprets responses from** the LLM.

---

## ✅ How `AgentLanguage` Is Used in the Agent Loop

### 🔹 Step 1: Constructing the Prompt

When the agent loop begins, it first constructs the LLM prompt like this:

```python
def construct_prompt(self, goals: List[Goal], memory: Memory, actions: ActionRegistry) -> Prompt:
    return self.agent_language.construct_prompt(
        actions=actions.get_actions(),
        environment=self.environment,
        goals=goals,
        memory=memory
    )
```

This method builds a **structured input** for the model using four key components:

| Component      | Description                                                             |
| -------------- | ----------------------------------------------------------------------- |
| 🧭 Goals       | What the agent is trying to accomplish                                  |
| 🛠️ Actions    | The tools or functions available to the agent                           |
| 🧠 Memory      | Prior messages, file contents, and context relevant to the current task |
| 🌍 Environment | Constraints, settings, or metadata about where the agent is running     |

---

## 🔄 Parsing Responses: AgentLanguage Again

Later in the loop, after receiving the LLM response, `AgentLanguage` also handles **interpreting that response**:

```python
action, invocation = self.agent_language.parse_action_response(response)
```

If you're using **OpenAI function calling**, this is typically as simple as extracting the structured `tool_calls` block.

But by **decoupling** the formatting/parsing logic into `AgentLanguage`, you can:

* 🔁 Swap out OpenAI for another LLM format
* 📜 Use natural language responses instead of tool calls (for simulation)
* 🧪 Test your agent without hard-coding the LLM's output logic

---

## 🧩 Why This Matters

This modular `AgentLanguage` component lets your agent:

* Speak **different languages** (OpenAI function calling, plain text, etc.)
* Be tested or simulated in different contexts
* Separate **communication logic** from **business logic**

So even though we use OpenAI tools most of the time, this abstraction gives you a path to scale, extend, and simulate more flexibly.

---


## 🧠 **Memory Context**

**Definition**:
Memory is a record of everything the agent has "seen" or "done" so far in the current session.

### 🔍 Why it matters:

* It helps the agent **maintain continuity** over multiple steps.
* Without it, the agent would be stateless — it would forget past actions, results, or user instructions.

### 🧩 What's typically stored:

* The **user’s original request**
* Past **LLM responses**
* Previous **tool invocations**
* Results of those tools (e.g., content from a file it read)

### ✅ Example:

```python
[
  {"type": "user", "content": "Refactor the data processing script."},
  {"type": "assistant", "content": "I'll begin by listing files in the 'scripts/' directory."},
  {"type": "user", "content": "['clean.py', 'process.py']"},
  {"type": "assistant", "content": "Reading process.py now."},
  {"type": "user", "content": "# contents of process.py..."}
]
```

This history allows the agent to ask follow-up questions, avoid repeating steps, and remember relevant files or outputs.

---

## 🌍 **Environment Info**

**Definition**:
A description of the **context** in which the agent is operating. This is static info about the world the agent is working in.

### 📦 Typical contents:

* "You are working in a local Python project folder."
* "You have access to functions like `list_files()` and `read_file()`."
* "You do *not* have internet access."

### 🔐 Why it matters:

It **guides the agent’s reasoning boundaries**:

* Prevents hallucinating actions it cannot perform (like calling an API when it's not available).
* Sets correct assumptions about what data/tools are available.

### ✅ Example:

```text
You are operating in a local development environment.
You can read and write files, but cannot access the internet.
Use the tools provided below to complete the task.
```

---

## 💡 Summary

| Component        | Purpose                                           | Keeps Agent Aware Of     |
| ---------------- | ------------------------------------------------- | ------------------------ |
| Memory Context   | Conversation and execution history                | What’s already been done |
| Environment Info | Operational boundaries and available capabilities | What’s *possible* to do  |



### 🤖 Step 2: Generating a Response

After the prompt is constructed, the agent sends it to the language model:

```python
def prompt_llm_for_action(self, full_prompt: Prompt) -> str:
    response = self.generate_response(full_prompt)
    return response
```

### 🔧 What's Happening?

* `generate_response()` is an **injected function** defined during initialization.
* It handles the actual call to the LLM.
* This abstraction allows the agent framework to stay **model-agnostic**.

---

### 🤖 Why This Matters

This design provides flexibility:

* You can use **LiteLLM**, **OpenAI**, **Anthropic**, or any LLM.
* You don’t have to change the core agent loop to switch models.
* Great for mocking, testing, or deploying in environments with different LLMs.


### 🧠 Step 3: Parsing the Response

```python
action, invocation = self.get_action(response)
```

* `AgentLanguage` parses the model’s reply.
* `ActionRegistry` retrieves the action definition.
* `invocation` contains the tool name and argument values.

---

## ✅ What Is `AgentLanguage`?

`AgentLanguage` is a **custom abstraction** used in this lecture's framework to:

> Define how the LLM communicates actions to the agent.

It **controls how the LLM should express tool calls** in text, and how we **parse** those calls from its output.

### Think of `AgentLanguage` as:

* A bridge between **natural language** (from the LLM) and **structured commands** (the agent executes).
* A reusable module that **parses the LLM's output** into something your code can understand and act on.

### It likely provides:

* `format_action(action_name, args)` → Returns a prompt string to tell the LLM how to call the tool.
* `parse_action_response(response_text)` → Extracts `action_name` and `args` from the LLM’s text.

---

## 🧠 So What Is `get_action()` Doing?

In the agent loop:

```python
action, invocation = self.get_action(response)
```

Here’s what’s happening:

1. **`response`** = the raw LLM output (text).
2. `self.get_action()` internally calls something like:

   ```python
   return self.language.parse_action_response(response)
   ```
3. That parsing breaks the response into:

   * `action`: the name of the tool (e.g., `"read_python_file"`)
   * `invocation`: the actual argument dictionary (e.g., `{"file_name": "main.py"}`)

So **you don’t see `parse_response()` defined**, because it's likely wrapped inside this `AgentLanguage` object’s method.

---

## 🔁 Where Does This Fit in the Loop?

The structure is roughly:

```python
prompt = self.construct_prompt(...)
response = llm_call(prompt)
action, invocation = agent_language.parse_action_response(response)
tool_fn = registry[action]
tool_fn(**invocation)
```

---

## 📌 Summary

| Term            | Role                                                    |
| --------------- | ------------------------------------------------------- |
| `AgentLanguage` | Class that defines LLM input/output formatting          |
| `get_action()`  | Uses `AgentLanguage` to extract tool call from response |
| `invocation`    | Parsed tool arguments ready to call in Python           |





### 🔧 Step 4: Executing the Action

Once the tool and arguments are known, the agent performs the action:

```python
result = self.environment.execute_action(action, invocation["args"])
```

### 🛠️ Execution Happens in the Environment

* The `Environment` class is responsible for **carrying out the action**.
* This might involve:

  * Calling APIs
  * Accessing files
  * Running computations
  * Querying databases

### 🧩 Separation of Concerns

* The `ActionRegistry` knows **what** can be done.
* The `Environment` knows **how** to do it in the current context.







### 🧠 Step 5: Updating Memory

After the agent executes an action, it needs to **record what happened**—both the decision it made and the result of that action.

```python
def update_memory(self, memory: Memory, response: str, result: dict):
    """
    Update memory with the agent's decision and the environment's response.
    """
    new_memories = [
        {"type": "assistant", "content": response},             # What the LLM said
        {"type": "user", "content": json.dumps(result)}         # What the environment returned
    ]
    for m in new_memories:
        memory.add_memory(m)
```

### 💡 Why Update Memory?

* Keeps a **chronological record** of what the agent did and why.
* Memory becomes **part of the next prompt**, allowing the agent to reason across multiple steps.
* Helps the agent **avoid repetition**, reuse past insights, and improve coherence.

### 🧠 What Gets Stored?

1. The **LLM’s response** (usually a tool call or text-based reasoning).
2. The **environment’s result** (i.e., output from the executed action).

Together, this builds a conversational history that gets looped back into the LLM’s input, making the agent smarter over time.

---

### ✅ Here's what happens regarding token usage and cost:

#### 💾 1. **Memory is added to the prompt**

Each time the agent loops:

* It **constructs a new prompt**.
* This prompt includes:

  * The agent’s goals.
  * Available tool definitions (tool schemas).
  * The full **memory history** so far (unless truncated).
  * Possibly some environmental context.

#### 🔄 2. **All of that is sent to the LLM**

* OpenAI (and most LLM APIs) **charge based on token input and output**.
* So every piece of memory stored — whether it’s the user's request, the agent's tool call, or the tool's result — will be **converted into tokens** and **counted toward the prompt token budget**.

#### 💸 3. **You pay for it**

* You're billed for:

  * All **tokens in the request** (the full prompt).
  * All **tokens in the response** (e.g., a tool call or text reply).
* In long-running loops, the **memory can grow large**, increasing prompt size and cost quickly.

---

### 🧠 How to Manage This

* 🔄 **Summarize or compress memory**: Store only the essential decisions and results.
* 📉 **Truncate old context**: Only keep the last N exchanges.
* 🧱 **Structured memory**: Instead of dumping raw results, store simplified records (e.g., `"Searched files A, B, found match in B"`).
* 🧠 **Hybrid memory**: Use a vector store or database to retrieve memory selectively, instead of including all memory in every call.




### ⛔ Step 6: Termination Check

```python
if self.should_terminate(response): break
```

* Uses `action_def.terminal` to see if the agent should stop.


---

### 🧠 Why We Need a Termination Check

Agents usually run in loops:

1. Construct a prompt.
2. Call the LLM.
3. Decide what to do.
4. Execute a tool.
5. Update memory.
6. Repeat...

But this can’t go on forever. At some point, the agent must decide:

> “I’ve done everything I needed to do. Time to stop.”

That’s where the **`should_terminate()`** check comes in.

---

### 🔍 What the Code Does

```python
def should_terminate(self, response: str) -> bool:
    action_def, _ = self.get_action(response)
    return action_def.terminal
```

Here’s what this means, step by step:

1. **`get_action(response)`**
   → This parses the model’s response and returns:

   * `action_def`: the tool definition (from the registry)
   * `invocation`: the tool name + arguments

2. **`action_def.terminal`**
   → This is a Boolean property on the tool definition:

   * If `True`, this tool is a **terminal action**, i.e. “I'm done.”
   * If `False`, the loop should continue.

---

### ✅ Example: “terminate\_agent” Tool

You might define a tool like this:

```python
Action(
    name="terminate_agent",
    description="Indicates that the task is complete and the agent should shut down.",
    parameters={},
    terminal=True  # <--- THIS is what causes the loop to end
)
```

When the model selects this action, `should_terminate()` will return `True`, and the loop exits.

---

### 🧠 Why This Design Is Powerful

* ✅ It gives the **LLM control** over when it's done — like finishing a conversation.
* ✅ It keeps the agent loop generic — you don’t hardcode “when” to stop.
* ✅ You can have **multiple terminal tools**, like `terminate`, `cancel`, or `complete_task`.















## 🔁 Information Flow in One Loop Iteration

1. **Memory**: Supplies past conversations and results.
2. **Goals**: Define what the agent is trying to achieve.
3. **ActionRegistry**: Tells what the agent *can* do.
4. **AgentLanguage**: Converts everything into a structured prompt.
5. **LLM**: Chooses an action and returns tool + args.
6. **Environment**: Executes the selected action.
7. **Memory**: Gets updated with the result and decision.
8. **Loop**: Repeats or ends.

---

## 🧪 Creating Specialized Agents (Examples)

### 📚 Research Agent

```python
research_agent = Agent(
    goals=[Goal("Find and summarize information on topic X")],
    agent_language=ResearchLanguage(),
    action_registry=ActionRegistry([SearchAction(), SummarizeAction(), ...]),
    generate_response=openai_call,
    environment=WebEnvironment()
)
```

### 🧑‍💻 Coding Agent

```python
coding_agent = Agent(
    goals=[Goal("Write and debug Python code for task Y")],
    agent_language=CodingLanguage(),
    action_registry=ActionRegistry([WriteCodeAction(), TestCodeAction(), ...]),
    generate_response=anthropic_call,
    environment=DevEnvironment()
)
```

Each agent uses the **same loop** but behaves entirely differently due to their unique GAME components.

